In [1]:
import numpy as np
from pathlib import Path

from tarea2.algorithms import simplex_normalization, north_west_heuristic, costos_reducidos
from tarea2.io import get_data
from tarea2.gen import gen_A , gen_b, gen_mapping

data_path = Path("data")
#data_path = Path("data_reduced")
#data_path = Path("northwest_example")

# Obtener los datos

In [2]:
I, J, oferta_i, demanda_j, costos_ij = get_data(data_path)

print(f"{oferta_i.shape = }")
print(f"{demanda_j.shape = }")
print(f"{costos_ij.shape = }")


A = gen_A(I, J)
b = gen_b(oferta_i, demanda_j)
#mapping = gen_mapping(I, J)

print(f"{A.shape = }")
print(f"{b.shape = }")
#print(f"{mapping.shape = }")

El archivo alpha_i.csv existe :)
El archivo beta_j.csv existe :)
El archivo a_ij.csv existe :)
oferta_i.shape = (20, 1)
demanda_j.shape = (30, 1)
costos_ij.shape = (20, 30)
A.shape = (49, 600)
b.shape = (49, 1)


# Generar solución factible inicial

In [3]:
solucion = north_west_heuristic(oferta_i, demanda_j, costos_ij)

In [4]:
print(solucion)
print(costos_ij)
print((solucion * costos_ij).sum())

[[3306.  350.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0. 1009. 2652.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0.    0.  140. 2417. 1048.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  723. 2907.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0. 1035. 3346.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.  954. 2516.  593.    0.    

# Generar índices de base

In [5]:
X = solucion.ravel(order="C")
c = costos_ij.ravel()

idx_B = [idx for idx, x in enumerate(X) if x != 0]
idx_NB = [idx for idx, x in enumerate(X) if x == 0]

In [6]:
print(len(X))
print(len(X[idx_B]))

600
49


In [7]:
print(A.shape)
print(X[idx_B].shape)

#np.dot(A,solucion_flatten[base_i].T)

(49, 600)
(49,)


In [8]:
X_B = X[idx_B]
c_B = c[idx_B]
A_B = A[:, idx_B]

X_NB = X[idx_NB]
c_NB = c[idx_NB]
A_NB = A[:, idx_NB]

print(f"{X_B.shape = }")
print(f"{c_B.shape = }")
print(f"{A_B.shape = }")

print(f"{X_NB.shape = }")
print(f"{c_NB.shape = }")
print(f"{A_NB.shape = }")

print(f"La suma de X_nobase debe ser 0: {X[idx_NB].sum()}")

X_B.shape = (49,)
c_B.shape = (49,)
A_B.shape = (49, 49)
X_NB.shape = (551,)
c_NB.shape = (551,)
A_NB.shape = (49, 551)
La suma de X_nobase debe ser 0: 0.0


Todos los elementos de la solución base sean positivos para ser **solución básica factible**

In [9]:
(X_B > 0).all()

True

Para ser solución base, el producto con la inversa de A_B debe ser b
$$
A_B x_B = b
\\
 x_B = A_B^{-1} b
$$

In [10]:
print(np.dot( A_B, X_B))
print(b.T)


print((np.dot( A_B, X_B) == b.T).all())

[3656. 3661. 3605. 3630. 4381. 4063. 4159. 3763. 4286. 3939. 4297. 3521.
 3691. 4344. 3839. 4396. 4159. 3998. 3869. 3559. 3306. 1359. 2792. 2417.
 1771. 3942. 4300. 2516. 3276. 2621. 3728. 4586. 1465. 1626. 1737. 4345.
 3405. 1772. 2144. 1274. 2683. 1920. 1206. 1201. 1454. 1226. 3261. 4208.
 4261.]
[[3656 3661 3605 3630 4381 4063 4159 3763 4286 3939 4297 3521 3691 4344
  3839 4396 4159 3998 3869 3559 3306 1359 2792 2417 1771 3942 4300 2516
  3276 2621 3728 4586 1465 1626 1737 4345 3405 1772 2144 1274 2683 1920
  1206 1201 1454 1226 3261 4208 4261]]
True


# Costos Reducidos

In [11]:
cr = costos_reducidos(idx_B, idx_NB, A, c)

def es_factible(X_B):
  return (X_B >= 0).all()

def es_optimo(cr):

  if (cr >= 0).all():
    print("Tenemos un óptimo")

    if (cr == 0).any():
      print("Son múltiples óptimos")
    else:
      print("Es único")

    return True

  return False


if (es_factible(X_B)):
  es_optimo(cr)

Obtener la dirección básica N-ésima

In [26]:
d = np.zeros(shape=X.shape)
d_B = - np.dot( np.linalg.inv(A_B), A[:, idx_NB[1]] )
#d_NB = d[idx_NB]

#d_B[:] = 1
#d.sum()



d_B

array([-0., -1.,  1., -1.,  1., -1., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])